# Create matrix tables 

In this notebook, we prepare matrix tables for use in analysis.

# Setup 

In [ ]:
from datetime import datetime
import hail as hl
import os
import time

## Retrieve exome region file

In [ ]:
!wget -nd biobank.ndph.ox.ac.uk/ukb/ukb/auxdata/xgen_plus_spikein.GRCh38.bed

In [ ]:
!gsutil cp xgen_plus_spikein.GRCh38.bed ${WORKSPACE_BUCKET}/data/ukb/exomes/

## Define constants

In [ ]:
AOU_VCFS = 'gs://fc-aou-preprod-datasets-controlled/5/wgs/vcf/merged/alpha2/*.vcf.gz'
UKB_VCFS = 'gs://fc-7130e767-a885-4678-95ed-7c966c79e2d0/200K/pvcf/ukb23156_*.vcf.gz'
EXOME_REGIONS = f'{os.getenv("WORKSPACE_BUCKET")}/data/ukb/exomes/xgen_plus_spikein.GRCh38.bed'

INPUT_VCFS = AOU_VCFS

In [ ]:
RESULT_BUCKET = os.getenv("WORKSPACE_BUCKET")
DATESTAMP = time.strftime('%Y%m%d')
TIMESTAMP = time.strftime('%Y%m%d_%H%M%S')
WORK_DIR = !pwd

# Output files
OUTPUT_MT = f'{os.getenv("WORKSPACE_BUCKET")}/data/aou/alpha2/cohort.mt'
HAIL_LOG = f'{WORK_DIR[0]}/hail-make-mt-{TIMESTAMP}.log'
HAIL_LOG_DIR_FOR_PROVENANCE = f'{os.getenv("WORKSPACE_BUCKET")}/hail-logs/{DATESTAMP}/'

## Check access

In [ ]:
!gsutil ls {AOU_VCFS} | head

In [ ]:
!gsutil ls {UKB_VCFS} | head

# Start Hail 

In [ ]:
# See also https://towardsdatascience.com/fetch-failed-exception-in-apache-spark-decrypting-the-most-common-causes-b8dff21075c
# See https://spark.apache.org/docs/2.4.7/configuration.html

EXTRA_SPARK_CONFIG = {
    # If set to "true", performs speculative execution of tasks. This means if one or more tasks are running
    # slowly in a stage, they will be re-launched.
    'spark.speculation': 'true', # Default is false.
    
    # Fraction of tasks which must be complete before speculation is enabled for a particular stage.
    'spark.speculation.quantile': '0.95', # Default is 0.75

    # Default timeout for all network interactions. This config will be used in place of 
    # spark.core.connection.ack.wait.timeout, spark.storage.blockManagerSlaveTimeoutMs, 
    # spark.shuffle.io.connectionTimeout, spark.rpc.askTimeout or spark.rpc.lookupTimeout if they are not configured.
    'spark.network.timeout': '180s', # Default is 120s
        
    # (Netty only) Fetches that fail due to IO-related exceptions are automatically retried if this is set to a
    # non-zero value. This retry logic helps stabilize large shuffles in the face of long GC pauses or transient
    # network connectivity issues.
    'spark.shuffle.io.maxRetries': '10',  # Default is 3
    
    # (Netty only) How long to wait between retries of fetches. The maximum delay caused by retrying is 15 seconds
    # by default, calculated as maxRetries * retryWait.
    'spark.shuffle.io.retryWait': '15s',  # Default is 5s
    
    # Number of failures of any particular task before giving up on the job. The total number of failures spread
    # across different tasks will not cause the job to fail; a particular task has to fail this number of attempts.
    # Should be greater than or equal to 1. Number of allowed retries = this value - 1.
    'spark.task.maxFailures': '10', # Default is 4.

    # Number of consecutive stage attempts allowed before a stage is aborted.
    'spark.stage.maxConsecutiveAttempts': '10' # Default is 4.
}

In [ ]:
hl.init(spark_conf=EXTRA_SPARK_CONFIG,
        min_block_size=50,
        default_reference='GRCh38',
        log=HAIL_LOG)

Check the configuration.

In [ ]:
sc = hl.spark_context()
config = sc._conf.getAll()
config.sort()
config

# Load exome capture regions

In [ ]:
ukb_exome_capture_regions = hl.import_bed(EXOME_REGIONS)

In [ ]:
ukb_exome_capture_regions.describe()

In [ ]:
ukb_exome_capture_regions.show(5)

# Create matrix table from VCFs

In [ ]:
mt = hl.import_vcf(INPUT_VCFS,
                   array_elements_required=False,
                   force_bgz=True)

In [ ]:
mt.describe()

In [ ]:
mt = mt.filter_rows(
    hl.is_defined(ukb_exome_capture_regions[mt.locus]))

In [ ]:
start = datetime.now()
print(start)

In [ ]:
mt.write(OUTPUT_MT, overwrite=True)

In [ ]:
end = datetime.now()
print(end)
print(end - start)

In [ ]:
!gsutil ls {OUTPUT_MT}

# Provenance

In [ ]:
# Copy the Hail log to the workspace bucket so that we can retain it.
!gzip --keep {HAIL_LOG}
!gsutil cp {HAIL_LOG}.gz {HAIL_LOG_DIR_FOR_PROVENANCE}

In [ ]:
print(datetime.now())

In [ ]:
!pip3 freeze